# Refinitiv Data Library for Python
## Content - Pricing - Streaming

This notebook demonstrates how a Price object to receive streaming real-time data.  

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook - which I have done, and so I don't need to specify a session_type:

In [2]:
# I have defined a default Session type in the Credentials notebook - so no need to provide one here
open_session()

### Define a callback to receive streaming data

In [3]:
# declare a DataFrame to use for storing our streaming data
df = DataFrame()
# and one to store status events
df_status = DataFrame()

# Function to handle the intial Refresh for each item
def handle_refresh(streaming_prices, instrument_name, fields):
# We will do nothing in this instance - instead  we will use the create_dataframe below
# to create a dataframe - once we have received initial values for all items
    return

# Function to initially populate Dataframe, once initial values received for all items
def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    df['Status']=''
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
        if price.name in df_status.index:
            df.at[price.name, 'Status'] = df_status.code[price.name]
    display(df)

# Function to update dataframe, when we receive updates for individual items
def update_dataframe(streaming_prices, instrument_name, fields):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    if instrument_name in df_status.index:
            df.at[instrument_name, 'Status'] = df_status.code[price.name]
    display(df)
    
# Function to extract status code for an item as & when received from server
# The status object also contains a more detailed 'message'
def get_status(streaming_prices, instrument_name, status):
    global init_status
    df_status.at[instrument_name, 'code'] = status['code']

### Create streams for our RICs and open them

In [4]:
# Define our Streaming Price object
streams = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()

# Callback for if we wanted to handle invidiual Refresh for each item
streams.on_refresh(handle_refresh)
# Callback for when we have received initial values for all items
streams.on_complete(create_dataframe)
# Specify callback handler for any updates
streams.on_update(update_dataframe)
# Specify callback handler for any updates
streams.on_status(get_status)

# Send the requests to the server and open the streams for all items
streams.open()

,BID,ASK,Status
EUR=,1.1901,1.1905,
GBP=,1.3915,1.3919,
JPY=,110.76,110.79,
CAD=,1.2371,1.2375,


## Close the stream

In [ ]:
streams.close()

## Close the session

In [ ]:
close_session()